In [ ]:
from pprint import pprint

import pandas as pd

from aika import putki
from aika.putki import CalendarChecker
from aika.putki.context import Defaults, GraphContext
from aika.putki.graph import Graph, TaskModule
from aika.putki.runners import LocalRunner
from aika.putki.interface import Dependency
from aika.time.calendars import TimeOfDayCalendar
from aika.time.time_of_day import TimeOfDay
from aika.time.time_range import TimeRange#
from aika.time.timestamp import Timestamp
from aika.utilities.fin.macd import macd

from aika.datagraph.persistence.hash_backed import HashBackedPersistanceEngine
from aika.datagraph.persistence.mongo_backed import MongoBackedPersistanceEngine
from pandas_datareader import data
import typing as t
from pandas.tseries.offsets import BDay, CDay
import pymongo

## Create your first function. 
This just uses pandas datareader project to pull some stock data from yahoo. 

In [ ]:
tod = TimeOfDay.from_str("16:30:00 [America/New_York]")

def pull_google_finance_data(
    tickers : t.List,
    time_range,
):
    df = data.DataReader(list(tickers), "yahoo", start=time_range.start, end=time_range.end)
    df.index.name = None
    df.index = df.index.map(tod.make_timestamp) # this ensures it has a timezone.
    return df["Adj Close"]

In [ ]:
df = pull_google_finance_data(["AAPL", "GOOGL"], TimeRange("2018", "2020"))
df

Note the missing days, hour holiday calendar.

In [ ]:
holidays = pd.bdate_range(start="2018", end="2020").difference(df.index.date)
holidays

In [ ]:
context_ends_trading_day = GraphContext(
    defaults=Defaults(
        version="research", 
        persistence_engine=HashBackedPersistanceEngine(), 
        time_range= TimeRange("2018", "2020")
    )
)

context_ends_holiday = GraphContext(
    defaults=Defaults(
        version="research2", 
        persistence_engine=HashBackedPersistanceEngine(), 
        time_range= TimeRange("2018", "2019-12-26")
    )
)

In [ ]:
close_prices_broken = context_ends_holiday.time_series_task(
    "close_prices",
    pull_google_finance_data,
    tickers=("AAPL", "GOOGL"),
    completion_checker=CalendarChecker(
        TimeOfDayCalendar(time_of_day=tod, freq=BDay())
    )
)
close_prices_broken.run()

Note the above error message, the task did write output, but as it did not know that it was a holiday it appears to be missing a day, it was expecting an entry on christmas day, but if we read it we see:

In [ ]:
close_prices_broken.read().tail()

To fix this, we need only add a holiday calendar:

In [ ]:
close_prices_fixed = context_ends_holiday.time_series_task(
    "close_prices_fixed",
    pull_google_finance_data,
    tickers=("AAPL", "GOOGL"),
    completion_checker=CalendarChecker(
        TimeOfDayCalendar(time_of_day=tod, freq=CDay(weekmask="1111100", holidays=holidays))
    )
)
close_prices_fixed.run()
close_prices_fixed.read().tail()

Note that both of these tasks wrote the same data, just one correctly knows not to expect data on the day in question.